In [25]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import pandas as pd 

In [7]:
%%capture
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Cache is valid for [23/04/2023]
[WDM] - Looking for [chromedriver 112.0.5615.49 linux64] driver in cache 
[WDM] - Driver found in cache [/home/harry/.wdm/drivers/chromedriver/112.0.5615.49/linux64/chromedriver]


In [8]:
search_url="https://aiseo.ai/tools/paraphrasing-tool.html"
driver.get(search_url)

In [299]:
def gather_data(dataframe, driver, nrepeat = 2, timelapse = 5):
    
    testing = driver.find_element_by_css_selector('div.lmt__side_container.lmt__side_container--source')
    testing2 = testing.find_element_by_css_selector('div.ce-block__content')
    testing3 = testing2.find_element_by_xpath("//div[contains(text(),'dummy')]")
    testing3.clear()
    data = dict()
    for i in range(dataframe.shape[0]):
        record = list()
        for n in range(nrepeat):
            features = ', '.join(dataframe.iloc[i])
            inputs = 'galaxy, ' + features
            testing3.send_keys(inputs)       
            button = testing.find_element_by_xpath("//button[@id='aiseo-generate-btn']")
            button.click()
            time.sleep(timelapse)
            testing4 = driver.find_element_by_css_selector('div.lmt__side_container.lmt__side_container--target')
            testing5 = testing4.find_element_by_css_selector('div.ce-block__content')
            testing3.clear()
            record.append(testing5.text)
        data[i] = record
    return pd.DataFrame(data).T

In [297]:
dataframe = pd.DataFrame([['spiral', 'three arms', 'strong bar'], ['elliptical', 'round', 'elongated']])

In [298]:
dataframe

,0,1,2
0,spiral,three arms,strong bar
1,elliptical,round,elongated


In [304]:
data = gather_data(dataframe, driver, nrepeat=5, timelapse=5)

In [305]:
data.iloc[1].values

array(['The term "Galaxy" refers to a collection of stars, dust, and',
       'A galaxy can take different shapes such as elliptical, round or elongated.',
       'A galaxy can be classified as elliptical if its shape is round or elongated',
       'Galaxies can have different shapes, including elliptical, round, and elongated',
       'A galaxy can have various shapes, such as elliptical, round, or elong'],
      dtype=object)

In [306]:
data.iloc[0].values

array(['The galaxy is classified as a spiral galaxy with three predominant arms and a robust bar.',
       'The galaxy in question is a spiral galaxy with three prominent arms and a strong bar.',
       'The galaxy has a spiral structure with three arms and a robust bar at its center.',
       'The Milky Way is a spiral galaxy, consisting of three main arms and a strong bar in its center.',
       'The galaxy features a spiral structure encompassing three arms and a prominent bar.'],
      dtype=object)